In [1]:
import os

import mindspore
from mindspore.dataset import text, GeneratorDataset, transforms
from mindspore import nn, context

from mindnlp.engine import Trainer, Evaluator
from mindnlp.engine.callbacks import CheckpointCallback, BestModelCallback
from mindnlp.metrics import Accuracy

[WARNING] ME(43040:139946754213696,MainProcess):2023-12-05-09:23:19.106.543 [mindspore/run_check/_check_version.py:357] MindSpore version 2.2.10.20231204 and Ascend AI software package (Ascend Data Center Solution)version 7.0 does not match, the version of software package expect one of ['7.1']. Please refer to the match info on: https://www.mindspore.cn/install
[WARNING] ME(43040:139946754213696,MainProcess):2023-12-05-09:23:20.465.235 [mindspore/run_check/_check_version.py:375] MindSpore version 2.2.10.20231204 and "te" wheel package version 7.0 does not match. For details, refer to the installation guidelines: https://www.mindspore.cn/install
[WARNING] ME(43040:139946754213696,MainProcess):2023-12-05-09:23:20.467.360 [mindspore/run_check/_check_version.py:382] MindSpore version 2.2.10.20231204 and "hccl" wheel package version 7.0 does not match. For details, refer to the installation guidelines: https://www.mindspore.cn/install
[WARNING] ME(43040:139946754213696,MainProcess):2023-12

In [2]:
# prepare dataset
class SentimentDataset:
    """Sentiment Dataset"""

    def __init__(self, path):
        self.path = path
        self._labels, self._text_a = [], []
        self._load()

    def _load(self):
        with open(self.path, "r", encoding="utf-8") as f:
            dataset = f.read()
        lines = dataset.split("\n")
        for line in lines[1:-1]:
            label, text_a = line.split("\t")
            self._labels.append(int(label))
            self._text_a.append(text_a)

    def __getitem__(self, index):
        return self._labels[index], self._text_a[index]

    def __len__(self):
        return len(self._labels)

In [3]:
# download dataset
!wget https://baidu-nlp.bj.bcebos.com/emotion_detection-dataset-1.0.0.tar.gz -O emotion_detection.tar.gz
!tar xvf emotion_detection.tar.gz

--2023-12-05 09:23:25--  https://baidu-nlp.bj.bcebos.com/emotion_detection-dataset-1.0.0.tar.gz
Resolving baidu-nlp.bj.bcebos.com (baidu-nlp.bj.bcebos.com)... 36.110.192.178, 2409:8c04:1001:1002:0:ff:b001:368a
Connecting to baidu-nlp.bj.bcebos.com (baidu-nlp.bj.bcebos.com)|36.110.192.178|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1710581 (1.6M) [application/x-gzip]
Saving to: ‘emotion_detection.tar.gz’

emotion_detection.t 100%[===================>]   1.63M  5.75MB/s    in 0.3s    

2023-12-05 09:23:26 (5.75 MB/s) - ‘emotion_detection.tar.gz’ saved [1710581/1710581]

data/
data/test.tsv
data/infer.tsv
data/dev.tsv
data/train.tsv
data/vocab.txt


In [4]:
import numpy as np

def process_dataset(source, tokenizer, max_seq_len=64, batch_size=32, shuffle=True):
    column_names = ["label", "text_a"]
    
    dataset = GeneratorDataset(source, column_names=column_names, shuffle=shuffle)
    # transforms
    type_cast_op = transforms.TypeCast(mindspore.int32)
    def tokenize_and_pad(text):
        tokenized = tokenizer(text, padding='max_length', truncation=True, max_length=max_seq_len)
        return tokenized[0], tokenized[2]
    # map dataset
    dataset = dataset.map(operations=tokenize_and_pad, input_columns="text_a", output_columns=['input_ids', 'attention_mask'])
    dataset = dataset.map(operations=[type_cast_op], input_columns="label", output_columns='labels')
    # batch dataset
    dataset = dataset.batch(batch_size)

    return dataset

In [5]:
from mindnlp.transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-chinese')

In [6]:
dataset_train = process_dataset(SentimentDataset("data/train.tsv"), tokenizer)
dataset_val = process_dataset(SentimentDataset("data/dev.tsv"), tokenizer)
dataset_test = process_dataset(SentimentDataset("data/test.tsv"), tokenizer, shuffle=False)

In [7]:
dataset_train.get_col_names()

['input_ids', 'attention_mask', 'labels']

In [8]:
from mindnlp.transformers import MSBertForSequenceClassification

# set bert config and define parameters for training
model = MSBertForSequenceClassification.from_pretrained('bert-base-chinese', num_labels=3)
model.enable_recompute()

loss_fn = nn.CrossEntropyLoss()
optimizer = nn.Adam(model.trainable_params(), learning_rate=2e-5)

sys:1: ResourceWarning: Unclosed socket <zmq.Socket(zmq.PUSH) at 0x7f4692f56de0>
sys:1: ResourceWarning: Unclosed socket <zmq.Socket(zmq.PUSH) at 0x7f46600c1670>
The following parameters in checkpoint files are not loaded:
['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
The following parameters in models are missing parameter:
['classifier.weight', 'classifier.bias']


In [9]:
metric = Accuracy()
# define callbacks to save checkpoints
ckpoint_cb = CheckpointCallback(save_path='checkpoint', ckpt_name='bert_emotect', epochs=1, keep_checkpoint_max=2)
best_model_cb = BestModelCallback(save_path='checkpoint', ckpt_name='bert_emotect_best', auto_load=True)

trainer = Trainer(network=model, loss_fn=loss_fn, train_dataset=dataset_train,
                  eval_dataset=dataset_val, metrics=metric,
                  epochs=1, optimizer=optimizer, callbacks=[ckpoint_cb, best_model_cb],
                  jit=False)

trainer.set_amp(level='O1')

Trainer will use 'StaticLossScaler' with `scale_value=2 ** 10` when `loss_scaler` is None.


In [10]:
# start training
trainer.run(tgt_columns="labels")

The train will start from the checkpoint saved in 'checkpoint'.


Epoch 0: 100%|██████████████████████████████████████▊| 301/302 [02:28<00:00,  4.02it/s, loss=0.37700403]

Epoch 0: 100%|███████████████████████████████████████| 302/302 [03:32<00:00,  1.42it/s, loss=0.37681282]


Checkpoint: 'bert_emotect_epoch_0.ckpt' has been saved in epoch: 0.


Evaluate:  97%|███████████████████████████████████████████████████████▎ | 33/34 [00:04<00:00,  8.51it/s]

Evaluate: 100%|█████████████████████████████████████████████████████████| 34/34 [00:07<00:00,  4.44it/s]


Evaluate Score: {'Accuracy': 0.9324074074074075}
---------------Best Model: 'bert_emotect_best.ckpt' has been saved in epoch: 0.---------------
Loading best model from 'checkpoint' with '['Accuracy']': [0.9324074074074075]...
---------------The model is already load the best model from 'bert_emotect_best.ckpt'.---------------


In [11]:
evaluator = Evaluator(network=model, eval_dataset=dataset_test, metrics=metric)
evaluator.run(tgt_columns="labels")

Evaluate:  97%|███████████████████████████████████████████████████████▎ | 32/33 [00:04<00:00,  7.83it/s]

Evaluate: 100%|█████████████████████████████████████████████████████████| 33/33 [00:07<00:00,  4.35it/s]

Evaluate Score: {'Accuracy': 0.890926640926641}


In [12]:
dataset_infer = SentimentDataset("data/infer.tsv")

In [13]:
def predict(text, label=None):
    label_map = {0: "消极", 1: "中性", 2: "积极"}

    text_tokenized = Tensor([tokenizer(text).input_ids])
    logits = model(text_tokenized)
    predict_label = logits[0].asnumpy().argmax()
    info = f"inputs: '{text}', predict: '{label_map[predict_label]}'"
    if label is not None:
        info += f" , label: '{label_map[label]}'"
    print(info)

In [14]:
from mindspore import Tensor

for label, text in dataset_infer:
    predict(text, label)

inputs: '我 要 客观', predict: '中性' , label: '中性'
inputs: '靠 你 真是 说 废话 吗', predict: '消极' , label: '消极'
inputs: '口嗅 会', predict: '中性' , label: '中性'
inputs: '每次 是 表妹 带 窝 飞 因为 窝路痴', predict: '中性' , label: '中性'
inputs: '别说 废话 我 问 你 个 问题', predict: '消极' , label: '消极'
inputs: '4967 是 新加坡 那 家 银行', predict: '中性' , label: '中性'
inputs: '是 我 喜欢 兔子', predict: '积极' , label: '积极'
inputs: '你 写 过 黄山 奇石 吗', predict: '中性' , label: '中性'
inputs: '一个一个 慢慢来', predict: '中性' , label: '中性'
inputs: '我 玩 过 这个 一点 都 不 好玩', predict: '消极' , label: '消极'
inputs: '网上 开发 女孩 的 QQ', predict: '中性' , label: '中性'
inputs: '背 你 猜 对 了', predict: '中性' , label: '中性'
inputs: '我 讨厌 你 ， 哼哼 哼 。 。', predict: '消极' , label: '消极'


In [15]:
predict("家人们咱就是说一整个无语住了 绝绝子叠buff")

inputs: '家人们咱就是说一整个无语住了 绝绝子叠buff', predict: '中性'
